In [7]:
import sys
sys.argv=[arg for arg in sys.argv if not arg.startswith('--')]
sys.path.append('.')

In [8]:
import numpy as nm
from argparse import ArgumentParser

from sfepy import data_dir
from sfepy.base.base import IndexedStruct
from sfepy.discrete import FieldVariable, Material, Integral, Function,Equation,Equations,Problem
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.discrete.conditions import Conditions, EssentialBC
from sfepy.terms import Term
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
from sfepy.mechanics.matcoefs import stiffness_from_lame

In [12]:
def shift_u_fun(ts,coors,bc=None,problem=None,shift=0.0):
    val=shift*coors[:,1]**2
    return val

def main():
    mesh=Mesh.from_file(data_dir + '/meshes/2d/rectangle_tri.mesh')
    domain=FEDomain('domain',mesh)

    min_x,max_x=domain.get_mesh_bounding_box()[:,0]
    eps=1e-8*(max_x-min_x)
    omega=domain.create_region('omega','all')
    gamma1=domain.create_region('gamma1','vertices in x < %.10f' %(min_x+eps),'facet')
    gamma2=domain.create_region('gamma2','vertices in x > %.10f' %(max_x-eps),'facet')

    field=Field.from_args('fu',nm.float64,'vector', omega,approx_order=2)
    u=FieldVariable('u','unknown',field)
    v=FieldVariable('v','test',field,primary_var_name='u')

    m=Material('m',D=stiffness_from_lame(dim=2,lam=1,mu=1))
    f=Material('f',val=[[0.02],[0.01]])

    integral=Integral('i',order=3)
    t1=Term.new('dw_lin_elastic(m.D,v,u)',integral,omega,m=m,v=v,u=u)
    t2=Term.new('dw_volume_lvf(f.val,v)',integral,omega,f=f,v=v)

    eq=Equation('balance',t1+t2)
    eqs=Equations([eq])

    fix_u=EssentialBC('fix_u',gamma1,{'u.all':0.0})
    bc_fun=Function('shift_u_fun',shift_u_fun,extra_args={'shift':0.01})
    shift_u=EssentialBC('shift_u',gamma2,{'u.0':bc_fun})

    ls=ScipyDirect({})
    nls_status=IndexedStruct()
    nls=Newton({},lin_solver=ls,status=nls_status)

    pb=Problem('elasticity',equations=eqs)
    pb.save_regions_as_groups('regions')
    pb.set_bcs(ebcs=Conditions([fix_u,shift_u]))
    pb.set_solver(nls)
    status=IndexedStruct()
    variable=pb.solve(status=status)
    pb.save_state('result1.vtk',variable)

main()



sfepy: reading mesh (C:\Users\yesda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sfepy/meshes/2d/rectangle_tri.mesh)...
sfepy:   number of vertices: 258
sfepy:   number of cells:
sfepy:     2_3: 454
sfepy: ...done in 0.00 s
sfepy: saving regions as groups...
sfepy:   omega
sfepy:   gamma1
sfepy:   gamma2
sfepy: ...done
sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (1815, 1815)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 39145 (1.19e+00% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     f
sfepy:     m
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 1.343114e+01 (rel: 1.000000e+00)
sfepy:   residual:    0.01 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.01 [s]
sfepy: nls: iter: 1, residual: 2.793967e-14 (rel: 2.080216e

Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy: solved in 1 steps in 0.03 seconds


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.